# Deadlines

In [ ]:
import json
import pandas as pd

In [ ]:
import utils.checker as check
import utils.presenter as show
import utils.schedule_interarrival as sit
import utils.schedule_solver__with_arrivals as ssv

In [ ]:
# Datei laden
with open("data/jobshop_instances.json", "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]
show.print_jobs(instance)

### a) Ankunftszeiten

In [ ]:
mean_interarrival_time = sit.calculate_mean_interarrival_time(instance)
mean_interarrival_time

In [ ]:
df_arrivals = sit.generate_job_arrivals_df_by_mean_interarrival_time(instance, t_a=mean_interarrival_time)
df_arrivals

### b) Schedule

In [ ]:
df_fcfs = ssv.schedule_fcfs_with_arrivals(instance, df_arrivals)
df_fcfs

In [ ]:
category = "First Come First Serve"

show.plot_gantt_jobs(df_fcfs, 'Gantt-Diagramm für "{}"'.format(category), duration_column= "Processing Time")
show.plot_gantt_machines(df_fcfs, 'Gantt-Diagramm für "{}"'.format(category), duration_column= "Processing Time")

check.check_all_constraints(df_fcfs, instance)

## Deadlines

$$
{d_{j}=a_{j}+k\,p_{j}}
$$  

$$
S(k)=\frac{1}{n}\sum_{j=1}^{n}\mathbf{1}\!\bigl[C_{j}(k)\le d_{j}(k)\bigr]
$$  

$$
S(k)\;\ge\;S_{\text{target}}
$$  

---

* **$a_{j}$** – Ankunftszeit (Release-Date) von Job $j$  
* **$p_{j}$** – *Processing time* des Jobs (aus dem Job Shop Scheduling-Problem) 

  $$p_{j}=\sum_{(m,d)\in j} d$$  

  Summe aller Operationsdauern $d$ von Job $j$ über sämtliche Maschinen $m$.  
* **$k$** – Due-Date-Faktor, den die Binärsuche einstellt  
* **$d_{j}$** – zugewiesene Deadline für Job $j$  
* **$C_{j}(k)$** – Fertigstellzeit von Job $j$ im geplanten Schedule bei Faktor $k$  
* **$\mathbf{1}[\;\cdot\;]$** – Indikatorfunktion (1, wenn Aussage wahr; sonst 0)  
* **$S(k)$** – gemessener Service-Level (Anteil pünktlicher Jobs) bei Faktor $k$  
* **$S_{\text{target}}$** – gewünschter Mindest-Service-Level (z. B. $0{,}95$ - mindestens $95$% aller **Jobs** sollen ihre **Deadline** einhalten)  


In [ ]:
def calc_due_dates(jobs: dict, arrivals: pd.DataFrame, k: float, buffer: float = 0.0) -> dict:
    """
    Berechnet Deadlines für jedes Job j als
      d_j = a_j + (k + buffer)*p_j
    """
    # Gesamtbearbeitungszeiten p_j
    p_tot = {j: sum(d for _, d in ops) for j, ops in jobs.items()}
    # Ankunftszeiten a_j
    a = arrivals.set_index("Job")["Arrival"].to_dict()
    # Deadline-Berechnung
    return {j: a[j] + (k + buffer) * p_tot[j] for j in jobs}


def find_k(jobs, arrivals, schedule_func, target_service=0.95, final_buffer: float = 0.0):
    """sucht k via Binärsuche; Schedule einmal vorterminiert"""
    
    # 1) einmaligen Schedule holen (für z.B. FCFS o.ä., der nicht deadline‐abhängig ist)
    sched = schedule_func(jobs, arrivals)
    
    # 2) Binärsuche
    lo, hi = 0.5, 5.0                      # Startintervall (evtl. anpassen)
    for _ in range(15):                    # 15 Iterationen ≈ 1/2^15 Genauigkeit            
        k = (lo + hi) / 2
        d = calc_due_dates(jobs, arrivals, k)
        
        # Service‐Level neu berechnen
        on_time = (sched["End"] <= sched["Job"].map(d)).mean()
        if on_time >= target_service:
            hi = k                          # Deadlines sind noch zu großzügig
        else:
            lo = k                          # Deadlines zu eng

    # Deadlines
    if final_buffer > 0:
        d = calc_due_dates(jobs, arrivals, k, buffer=final_buffer)
    return k, d

In [ ]:
k_opt, deadlines = find_k(instance, df_arrivals, ssv.schedule_fcfs_with_arrivals, target_service=0.95)
print(f"Gefundener Due-Date-Faktor k = {k_opt:.4f}")

In [ ]:
df_arrivals_deadlines = df_arrivals.assign(Deadline=df_arrivals["Job"].map(deadlines)).sort_values("Arrival")
df_temp= df_fcfs.groupby('Job', as_index=False).agg(End=('End', 'max'))
df_arrivals_deadlines.merge(df_temp, on="Job").sort_values("Arrival")[["Job","Arrival", "End", "Deadline"]]